<a href="https://colab.research.google.com/github/rafapcarvalho/imersao-ia-gemini/blob/main/chatBot_withDocuments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar google-generativeai

In [18]:
pip install google-generativeai

Instalar bibliotec para leitura de documento docx

In [19]:
pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.5 MB/s eta 0:00:00


Código de chat bot com respostas baseadas em documentos docx no repositório

In [28]:
from pathlib import Path
import hashlib
from docx import Document
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

def extract_docx_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF DOC ${pathname} ---"]
      # Add logic to read the DOCX and return a list of pages here.
  doc = Document(pathname)
  pages = []
  current_page = ""

  for paragraph in doc.paragraphs:
      if 'page break' in paragraph.text.lower():
          pages.append(current_page.strip())
          current_page = ""
      else:
          current_page += paragraph.text + "\n"

  if current_page:
      pages.append(current_page.strip())

  for index, page in enumerate(pages):
      parts.append(f"--- PAGE {index} ---")
      parts.append(page)
  return parts

chat = model.start_chat(history=[])

prompt = input('Você gostaria de saber sobre qual receita? ')

askWithExample = "\n\npergunta: Preciso de uma receita de pudim de leite condensado.\n\nresposta: No documento cujo nome do arquivo é Receitas Diversas.docx, você pode encontrar uma receita de pudim de leite condensado. O título da receita é Pudim de leite condensado simples, se encontra na página 3, e precisa de apenas 5 ingredientes.\n\n---\npergunta: receita de pudim.\n\nresposta: Desculpe não encontrei um receita cujo nome seja pudim, porém tenho uma de pudim de leite condensado, gostaria desta receita?\npergunta: sim\nresposta: No documento cujo nome do arquivo é Receitas Diversas.docx, você pode encontrar uma receita de pudim de leite condensado. O título da receita é Pudim de leite condensado simples, se encontra na página 3, e precisa de apenas 5 ingredientes.\n\n\n---\n\npergunta: Gostaria de uma receita de pamonha doce. pode me ajudar?\n\nresposta: No documento cujo nome do arquivo é Recitas Diversas.docx, você pode encontrar uma receita de pamonha doce. O título da receita é Pamonha doce, se encontra na página 4, e precisa de apenas 5 ingredientes. \n\n---\npergunta: Gostaria de uma receita de bolo de cenoura.\n\nresposta: No documento cujo nome do arquivo é Recitas Diversas.docx, você pode encontrar uma receita de bolo de cenoura. O título da receita é Bolo de Cenoura, se encontra na primeira página, e precisa de 9 ingredientes. \n\n---\n\npergunta: {} \n\nresposta:".format(prompt)

prompt_parts = [
  *extract_docx_pages("Receitas Diversas.docx"),
  *extract_docx_pages("Receitas Incriveis.docx"),
  askWithExample,
]

count = 0

while prompt != "fim":
  if count == 0:
    response = chat.send_message(prompt_parts)
    count += 1
  else:
    response = chat.send_message(prompt)

  print(response.text, '\n')
  prompt = input('')


Você gostaria de saber sobre qual receita? pamonha doce
No documento cujo nome do arquivo é Recitas Diversas.docx, você pode encontrar uma receita de pamonha doce. O título da receita é Pamonha doce, se encontra na página 4, e precisa de apenas 5 ingredientes. 
 

fim
